In [1]:
# Imports
from langchain import PromptTemplate
import os
import pprint

In [2]:
# Load API Keys
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Creating Wikipedia tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
# Creating Tavily tool

from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
web_search_tool = TavilySearchResults(k=3)

In [5]:
# Creating Web Page retrival tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="mxbai-embed-large"))
retriever=vectordb.as_retriever()

from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [6]:
# Creating a pdf retrival tool.
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('Pratiks March Resume.pdf')
docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs)
pdf_vectors = FAISS.from_documents(final_documents, OllamaEmbeddings(model="mxbai-embed-large"))
pdf_retriever=pdf_vectors.as_retriever()

from langchain.tools.retriever import create_retriever_tool
pdf_retriever_tool=create_retriever_tool(pdf_retriever,"pratik_resume_search",
                      "Search for information about Pratik Bhangale. For any questions about Pratik, you must use this tool!")


In [7]:
# Creating Arxiv tool

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

Tools are Wikipedia, Arxiv, Duckduckgo and Website Retriver

In [8]:
tools=[wiki,arxiv,retriever_tool, web_search_tool, pdf_retriever_tool]
print(tools)

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\prati\\anaconda3\\envs\\YTENV\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)), ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)), Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002BB856742C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], v

In [9]:
# Importing the Model and Prompt
from langchain_openai import ChatOpenAI
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [10]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
# Creating the agent

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
print(type(agent))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [12]:
# Creating the agent Executor

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
print(type(agent_executor))

<class 'langchain.agents.agent.AgentExecutor'>


In [13]:
# Asking questions using the invoke method.
response = agent_executor.invoke({"input":"What are the best research papers on LLMs?"})
pprint.pprint(response['output'])



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'LLMs'}`


Published: 2024-01-08
Title: A Survey of Large Language Models for Code: Evolution, Benchmarking, and Future Trends
Authors: Zibin Zheng, Kaiwen Ning, Yanlin Wang, Jingwen Zhang, Dewu Zheng, Mingxi YeOne of the best research papers on Large Language Models (LLMs) is titled "A Survey of Large Language Models for Code: Evolution, Benchmarking, and Future Trends" by Zibin Zheng, Kaiwen Ning, Yanlin Wang, Jingwen Zhang, Dewu Zheng, and Mingxi Ye. This paper provides insights into the evolution, benchmarking, and future trends of LLMs for code.

> Finished chain.
('One of the best research papers on Large Language Models (LLMs) is titled "A '
 'Survey of Large Language Models for Code: Evolution, Benchmarking, and '
 'Future Trends" by Zibin Zheng, Kaiwen Ning, Yanlin Wang, Jingwen Zhang, Dewu '
 'Zheng, and Mingxi Ye. This paper provides insights into the evolution, '
 'benchmarking, and future trends of 

In [14]:
# Asking questions using the invoke method.
response = agent_executor.invoke({"input":"What is the latest news?"})
pprint.pprint(response['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'latest news'}`


[{'url': 'https://www.nbcnews.com/', 'content': "judge asks in granting bail\nAsian America\nBrooklyn woman arrested after allegedly throwing hot coffee at a man in a Palestinian scarf\nU.S. news\nMichigan police make arrest in the disappearance of a woman missing since 2021\nAsian America\nBrooklyn woman arrested after allegedly throwing hot coffee at a man in a Palestinian scarf\nU.S. news\nMichigan police make arrest in the disappearance of a woman missing since 2021\nHealth\nHealth news\nMysterious dog respiratory illness may be caused by a new type of bacterial infection, researchers say\nCoronavirus\nFour more free Covid tests will be available to U.S. households\nHealth news\nThis type of belly fat is linked to increased risk of Alzheimer's, research finds\nHealth news\nNew weight loss drugs change how people think of Thanksgiving and other holiday meals\nHealth news

In [15]:
# Asking questions using the invoke method.
response = agent_executor.invoke({"input":"Who is the Prime minister of India?"})
pprint.pprint(response['output'])



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Prime Minister of India'}`


Page: Prime Minister of India
Summary: The prime minister of India (IAST: Bhārat kē Pradhānamantrī) is the head of government of the Republic of India. Executive authority is vested in the prime minisThe Prime Minister of India is the head of government of the Republic of India. The current Prime Minister of India is Narendra Modi.

> Finished chain.
('The Prime Minister of India is the head of government of the Republic of '
 'India. The current Prime Minister of India is Narendra Modi.')


In [16]:
# Asking questions using the invoke method.
response = agent_executor.invoke({"input":"What is Langchain?"})
pprint.pprint(response['output'])



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langchain'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

= traceable(async (user_input) => {    const result = await client.chat.completions.create({        messages: [{ role: "user", content: user_input }],        model: "gpt-3.5-turbo",    });    return result.choices[0].message.content;});await pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing in the how-to guides.5. Run your first evaluation​Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.create_dataset(dataset_name, description="A sample dataset in LangSmith.")client.creat

In [17]:
# Asking questions using the invoke method.
response = agent_executor.invoke({"input":"What are the skills of Pratik Bhangale?"})
pprint.pprint(response['output'])



> Entering new AgentExecutor chain...

Invoking: `pratik_resume_search` with `{'query': 'skills'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Datetime for project implementation.
•Conducted a comprehensive study on 10 prominent clustering algorithms and em-
ployed a systematic approach to implementing clustering algorithms involving steps
such as data extraction, manipulation, skewness reduction, normalization, model cre-
ation, and visualization.
•Concluded that 3 main algorithms (Kmeans, Mini-Batch Kmeans, and Agglomerative
clustering) are the most suitable for the given use-case based on visualization results.
•Model Training /external-link-altJan. 2021
Additional Experience And Awards
Technical Team Member (Association of Computer Science and Engineering Students) (2020 - 2021): Conducted
about 14 workshops and many technical treasure hunts for students in college. Certificate /external-link-alt
Top 20 Participant in HAB59 National Hackathon (2021) Participated as an individu